In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split

In [15]:
data = pd.read_csv("/kaggle/input/hackon-recommendation-data/user_payment_data.csv")

In [16]:
data.head()

,user_id,product_id,user_type,payment_method,price,cost,cashback,success_rate,user_history,label
0,1,1,cashback_oriented,Amazon Pay UPI,9184,0.039854,0.051408,0.958954,0,0
1,1,1,cashback_oriented,Google Pay,9184,0.034348,0.010926,0.909213,0,0
2,1,1,cashback_oriented,PhonePe,9184,0.027234,0.023123,0.920518,0,0
3,1,1,cashback_oriented,Credit Card,9184,0.014340,0.050905,0.949418,0,0
4,1,1,cashback_oriented,Debit Card,9184,0.016451,0.027689,0.866089,0,0


In [17]:


df = pd.DataFrame(data)

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        self.features = dataframe[['cashback', 'cost', 'success_rate', 'user_history']].values
        self.labels = dataframe['label'].values
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return feature, label

print("Data started")

# Create dataset and dataloaders
dataset = CustomDataset(df)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print("data created")

# Define the ANN model
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.layer1 = nn.Linear(4, 10)
        self.layer2 = nn.Linear(10, 5)
        self.layer3 = nn.Linear(5, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

print("model training")

# Initialize the model, loss function, and optimizer
model = ANNModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        labels = labels.unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Data started
data created
model training
Epoch [1/10], Loss: 0.0233
Epoch [2/10], Loss: 0.3885
Epoch [3/10], Loss: 0.2233
Epoch [4/10], Loss: 0.2803
Epoch [5/10], Loss: 0.0011
Epoch [6/10], Loss: 0.0114
Epoch [7/10], Loss: 0.0331
Epoch [8/10], Loss: 0.0001
Epoch [9/10], Loss: 0.0015
Epoch [10/10], Loss: 0.0002


In [18]:

# Evaluate the model
model.eval()
with torch.no_grad():
    mse = 0
    actuals = []
    predictions = []
    for features, labels in test_loader:
        labels = labels.unsqueeze(1)
        preds = model(features)
        mse += criterion(preds, labels).item()
        actuals.extend(labels.view(-1).tolist())  # Flatten the tensor and convert to list
        predictions.extend(preds.view(-1).tolist())  # Flatten the tensor and convert to list
    mse /= len(test_loader)
    print(f'Mean Squared Error: {mse:.4f}')

print(f'Actuals: {actuals[:20]}')
print(f'Predictions: {predictions[:20]}')

# Function to predict probability for new input
def predict_probability(model, scaler, cashback, cost, success_rate, user_history):
    model.eval()
    new_input = [[cashback, cost, success_rate, user_history]]
    new_input = scaler.transform(new_input)
    new_input_tensor = torch.tensor(new_input, dtype=torch.float32)
    with torch.no_grad():
        probability = model(new_input_tensor).item()
    return probability

# Example usage of the prediction function
cashback = 0.05
cost = 0.03
success_rate = 0.90
user_history = 0
predicted_prob = predict_probability(model, dataset.scaler, cashback, cost, success_rate, user_history)
print(f'Predicted Probability: {predicted_prob:.4f}')


Mean Squared Error: 0.0927
Actuals: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Predictions: [0.1914462298154831, 0.007642506621778011, 0.07172083854675293, 0.24953986704349518, 0.1380005180835724, 0.009094009175896645, 0.07260286062955856, 0.0702972263097763, 0.2971057593822479, 0.03778856620192528, 0.009958391077816486, 0.18511492013931274, 0.013028635643422604, 0.06482215225696564, 0.38244059681892395, 0.2146647721529007, 0.006777617149055004, 0.2588912546634674, 0.1518174260854721, 0.33782097697257996]
Predicted Probability: 0.0199


In [19]:
print(predict_probability( model , dataset.scaler , cashback = 0.1, cost = 0, success_rate =1 , user_history = 0.7))
print(predict_probability( model , dataset.scaler , cashback = 0 , cost = 0, success_rate =1 , user_history = 0.7))
print(predict_probability( model , dataset.scaler , cashback = 0.05, cost = 0, success_rate =1 , user_history = 0.7))

0.626914918422699
0.33476167917251587
0.4367274343967438


In [20]:
print(predict_probability( model , dataset.scaler , cashback = 0.1, cost = 0.02, success_rate =1 , user_history = 0.7))
print(predict_probability( model , dataset.scaler , cashback = 0 , cost = 0.02, success_rate =1 , user_history = 0.7))
print(predict_probability( model , dataset.scaler , cashback = 0.05, cost = 0, success_rate =1 , user_history = 0.7))

0.4100072979927063
0.04751351848244667
0.4367274343967438


In [21]:
torch.save(model , "ann_recommendation_model.pth")